In [ ]:
#We will use pandas for data cleaning
import pandas as pd

In [ ]:
raw_csv_data = pd.read_csv("Absenteeism-data.csv")

In [ ]:
type(raw_csv_data)

In [ ]:
#df = naming convention for Data Frame in pd.
#We create a copy of our initial csv file to make sure that we don't alter it.
df = raw_csv_data.copy()
df

In [ ]:
#We want to be able to see all columns and all rows, the entire dataframe , which is why we set the maximum value to None
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
display(df)

In [ ]:
df.info()

## 1. Drop the 'ID' column

In [ ]:
# .drop() method : We need to specify a column header ('ID') , and also that it's a column (axis = 1)
df = df.drop(['ID'], axis = 1)

## 2. 'Reason for Absence'

In [ ]:
df['Reason for Absence']

In [ ]:
#Extract the unique values of the column: equivalent to "df['Reason for Absence'].unique()""
pd.unique(df['Reason for Absence'])

In [ ]:
#We have 28 unique values while we have 29 reasons of absence : it means that a number between 0 and 28 is missing
len(df['Reason for Absence'].unique())

In [ ]:
#The value that we lack is 20 
sorted(df['Reason for Absence'].unique())

### 2.1 Create dummies with .get_dummies()

In [ ]:
reason_columns = pd.get_dummies(df['Reason for Absence'])
reason_columns

In [ ]:
#Create a new column called "check" that should be equal to 1 for each row.
reason_columns['check'] = reason_columns.sum(axis=1)
reason_columns['check']

In [ ]:
#700 = lenght of the dataframe
reason_columns['check'].sum(axis=0)

In [ ]:
#These 700 values are all equal to one = We only have one reason for absence per employee
reason_columns['check'].unique()

In [ ]:
reason_columns = reason_columns.drop(['check'], axis = 1)

In [ ]:
reason_columns = pd.get_dummies(df['Reason for Absence'], drop_first = True)
reason_columns

### 2.2 Group the reasons for absence in several dummy variables

In [ ]:
df.columns

In [ ]:
df.columns.values

In [ ]:
reason_columns.columns.values

In [ ]:
#To avoid multicollinearity, we drop the Reason for Absence column
df = df.drop(['Reason for Absence'], axis = 1)

In [ ]:
reason_columns.loc[:, 1:14]

In [ ]:
#For each row, we want to obtain the maximum value (0 or 1): Panda series
reason_columns.loc[:, 1:14].max(axis=1)

In [ ]:
#We allocate the different panda series to a variable
reason_type_1 = reason_columns.loc[:, 1:14].max(axis=1)
reason_type_2 = reason_columns.loc[:, 15:17].max(axis=1)
reason_type_3 = reason_columns.loc[:, 18:21].max(axis=1)
reason_type_4 = reason_columns.loc[:, 22:].max(axis=1)

### 2.3 Concatenate Column Values

In [ ]:
#We combine the objects horizontally (axis = 1)
df = pd.concat([df, reason_type_1, reason_type_2, reason_type_3, reason_type_4], axis = 1)
df

In [ ]:
df.columns

In [ ]:
#We give the reasons column names more explicit names (i-1 --> Reason_i)
column_names = ['Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours', 'Reason_1', 'Reason_2', 'Reason_3', 'Reason_4']

In [ ]:
df.columns = column_names
df.columns

In [ ]:
df.head()

### 2.4 Reorder Columns

In [ ]:
column_names_reordered = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 
                          'Date', 'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Absenteeism Time in Hours']

In [ ]:
#We use the list created above as an index, to have the reasons for absence at the front
df = df[column_names_reordered]

In [ ]:
df.head()

## 3. Create a Checkpoint

In [ ]:
df_reason_mod = df.copy()

In [ ]:
df_reason_mod

## 4. The 'Date' Column

### 4.1 Change of datatype

In [ ]:
#Structure of the date: Day/Month/Year
df_reason_mod["Date"]

In [ ]:
#Every column in a dataframe is in the form of a panda serie (we can index it)
type(df_reason_mod["Date"])

In [ ]:
#Type of each individual element in the serie: A string
type(df_reason_mod['Date'][0])

In [ ]:
#Converts the panda series into timestamp, by specifying the current format
df_reason_mod['Date'] = pd.to_datetime(df_reason_mod['Date'], format = '%d/%m/%Y')

In [ ]:
#New format: Year-Month-Day
df_reason_mod['Date']

In [ ]:
#New type of the individual element of the panda serie: Timestamp
type(df_reason_mod['Date'][0])

### 4.2 Extract the Month Value

In [ ]:
#With the timestamp datatype, we can extract the month directly by writing
# .month which take value from 1 to 12.
df_reason_mod['Date'][0].month

In [ ]:
list_months = []
list_months

In [ ]:
df_reason_mod.shape

In [ ]:
#Loop: for each row, we will append the monthly value obtained to the list created above
for i in range(df_reason_mod.shape[0]):
    list_months.append(df_reason_mod['Date'][i].month)

In [ ]:
list_months

In [ ]:
len(list_months)

In [ ]:
#We create a new column containing the elements in the list
df_reason_mod['Month Value'] = list_months

### 4.3 Extract the Day of the Week

In [ ]:
#Such as what we did with months, we can do it with days using weekday()
#values from 0 to 6: for Monday to Sunday
df_reason_mod['Date'][699].weekday()

In [ ]:
def date_to_weekday(date_value):
    return date_value.weekday()

In [ ]:
#The function will repeatedely apply to each element of the column that we mentionned
df_reason_mod['Day of the Week'] = df_reason_mod['Date'].apply(date_to_weekday)

In [ ]:
df_reason_mod.head()

### 4.4 Drop the "Date Column"

In [ ]:
# Drop the date column
df_reason_mod = df_reason_mod.drop(['Date'], axis = 1)

In [ ]:
#We want to place the Month and Day of the week columns right after the 
#Reason columns
df_reason_mod.columns.values

In [ ]:
column_names_upd = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value', 'Day of the Week',
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children',
       'Pets', 'Absenteeism Time in Hours']

In [ ]:
#We change the order of the columns 
df_reason_mod = df_reason_mod[column_names_upd]
df_reason_mod.head()

## 5. Create a checkpoint

In [ ]:
df_reason_date_mod = df_reason_mod.copy()
df_reason_date_mod

## 6. 'Education'

In [ ]:
#Transform education into a dummy variable with the max function.
display(df_reason_date_mod)

In [ ]:
df_reason_date_mod['Education'].unique()


In [ ]:
#It make sense to  have 1 (high school) alone since it has 583 appearances alone 
#,and the three other values (2,3,4) togheter in a category
df_reason_date_mod['Education'].value_counts()

In [ ]:
#.map({dictionary}): the old values will be the keys, and the values will be the new numbers
#0 : high school graduate (1)
#1 : graduate, postgraduate and master and doctor (2,3,4)
df_reason_date_mod['Education'] = df_reason_date_mod['Education'].map({1:0, 2:1, 3:1, 4:1})

In [ ]:
#We only have two categories now
df_reason_date_mod['Education'].unique()

In [ ]:
df_reason_date_mod['Education'].value_counts()

## Final Checkpoint

In [ ]:
df_cleaned = df_reason_date_mod.copy()
df_cleaned.head(10)